In [1]:
import os
import glob
import inro.modeller as m
import inro.emme.matrix as _matrix
import numpy as np
import pandas as pd
mm = m.Modeller()    
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
eb = mm.emmebank

In [2]:
scen_bau = 404

# scenario_2011 = 101
# scenario_2016 = 109
# scenario_2041 = 114

# ntwk_2011 = eb.scenario(scenario_2011).get_network()
# ntwk_2016 = eb.scenario(scenario_2016).get_network()
# ntwk_2041 = eb.scenario(scenario_2041).get_network()
            

In [3]:
### IMPORT ALL MATRICES IN SUBFOLDERS ###
def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)
def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"

In [4]:
#Matrix naming for inputs

# peak period - 7 hrs
sov_demand_pk = "TRIPS SOV peak"
hov2_demand_pk = "TRIPS HOV2 peak"
hov3_demand_pk = "TRIPS HOV3 peak"
bike_demand_pk = "TRIPS BIKE peak"
walk_demand_pk = "TRIPS WALK peak"
transit_demand_pk = "TRIPS TOTAL TRANSIT peak"

sov_demand_offpk = "TRIPS SOV offpeak"
hov2_demand_offpk = "TRIPS HOV2 offpeak"
hov3_demand_offpk = "TRIPS HOV3 offpeak"
bike_demand_offpk = "TRIPS BIKE offpeak"
walk_demand_offpk = "TRIPS WALK offpeak"
transit_demand_offpk = "TRIPS TOTAL TRANSIT offpeak"

In [5]:
#import all matrices from run output folder
hov2_oc = 2.2
hov3_oc = 3.3
#Delete all matrices
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"
local_transit_fare = "mf180"
transit_aux_time = "mf6"
truck_cost = "mf190"

matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Short List\BAU\BAU_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201127"
#scenario compatible with truck matrices
truck_scen = 401
scen_num = 404

clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_transit_fare,transit_aux_time,truck_cost],"FULL")

matrix_list = {
    sov_demand_pk : "",
    hov2_demand_pk: "",
    hov3_demand_pk : "",
    sov_demand_offpk : "",
    hov2_demand_offpk : "",
    hov3_demand_offpk : ""}

for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    print matrix_name
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                  matrix_description = matrix_name)
import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",scen_bau)

TRIPS HOV3 offpeak
TRIPS SOV peak
TRIPS HOV2 offpeak
TRIPS HOV3 peak
TRIPS SOV offpeak
TRIPS HOV2 peak
Imported matrix mf3
Imported matrix mf7
Imported matrix mf1
Imported matrix mf4
Imported matrix mf5
Imported matrix mf2
Finished Importing Matrices


In [6]:
#Calculate demand matrices

hov_dem_pk = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = hov_dem_pk,
              matrix_name="trips.peak.hov",
            matrix_description = "trips.peak.hov")
exp = matrix_list[hov2_demand_pk] + "/" + str(hov2_oc) + "+" + matrix_list[hov3_demand_pk] + "/" + str(hov3_oc)

spec = {
    "type": "MATRIX_CALCULATION",
    "result": hov_dem_pk,
    "expression": exp
}
matrix_calculator(spec)

hov_dem_offpk = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = hov_dem_offpk,
              matrix_name="trips.offpeak.hov",
            matrix_description = "trips.offpeak.hov")   
exp = matrix_list[hov2_demand_offpk] + "/" + str(hov2_oc)+ "+" + matrix_list[hov3_demand_offpk]+ "/" + str(hov3_oc)
spec = {
    "type": "MATRIX_CALCULATION",
    "result": hov_dem_offpk,
    "expression": exp
}
matrix_calculator(spec)

dem_pk = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id =dem_pk,
              matrix_name="trips.total",
            matrix_description = "trips.total")

exp = matrix_list[sov_demand_pk] + "+" + matrix_list[sov_demand_offpk] + "+" + hov_dem_offpk + "+" + hov_dem_pk

spec = {
    "type": "MATRIX_CALCULATION",
    "result": dem_pk,
    "expression": exp
}
matrix_calculator(spec)

{'average': 1.788957,
 'expression_type': 'FULL',
 'maximum': 14811.035156,
 'maximum_at': {'destination': 8885, 'origin': 8885},
 'minimum': 0.0,
 'minimum_at': {'destination': 1034, 'origin': 1001},
 'num_evaluations': 12327121,
 'result_type': 'FULL',
 'sum': 22052696.0}

In [7]:
indices = eb.matrix(dem_pk).get_data(scen_bau).indices
mat_np = eb.matrix(dem_pk).get_numpy_data(scen_bau)
df_full = pd.DataFrame(mat_np,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "O",
                                                                                           "level_1": "D",
                                                                                          0: "dem_pk"})

In [8]:
df_full.columns

Index([u'O', u'D', u'dem_pk'], dtype='object')

In [9]:
pth = r"C:\Users\PECHEN\Documents\Sketch Modelling"
megazones_df = pd.read_csv(os.path.join(pth,"TAZ_SketchModelZones_20201110.csv"))
df_int_ext = df_full[((df_full.O <= 9504) & (df_full.D <= 9504))]
df_int_ext = df_int_ext.merge(megazones_df, left_on = "O", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_O"})
df_int_ext = df_int_ext.merge(megazones_df, left_on = "D", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_D"})

In [10]:
df_int_ext.columns

Index([u'O', u'D', u'dem_pk', u'Megazone_O', u'Megazone_D'], dtype='object')

In [11]:
megazone_df = pd.pivot_table(df_int_ext,values = "dem_pk", index = ["Megazone_O","Megazone_D"],aggfunc = sum).reset_index()

In [12]:
megazone_df[(megazone_df.Megazone_O == "Extl West") | (megazone_df.Megazone_O == "Extl North") | (megazone_df.Megazone_O == "Extl South")| (megazone_df.Megazone_O == "Extl East")|(megazone_df.Megazone_D == "Extl West") | (megazone_df.Megazone_D == "Extl North") | (megazone_df.Megazone_D == "Extl South")| (megazone_df.Megazone_D == "Extl East")]["dem_pk"].sum()

0.0

In [13]:
megazone_df[(megazone_df.Megazone_O == "Extl West") | (megazone_df.Megazone_O == "Extl North") | (megazone_df.Megazone_O == "Extl South")| (megazone_df.Megazone_O == "Extl East")|(megazone_df.Megazone_D == "Extl West") | (megazone_df.Megazone_D == "Extl North") | (megazone_df.Megazone_D == "Extl South")| (megazone_df.Megazone_D == "Extl East")].to_csv(os.path.join(pth,"Megazone_External_OD_2051BAU_20201110.csv"))

In [14]:
megazone_df.to_csv(os.path.join(pth,"Megazone_External_OD_2051BAU_20210205.csv"))